In [3]:
import sys
sys.path.append("../decklizer")
from utils import *
from engine import *

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy

from IPython.display import clear_output
import time
import datetime
import re

from scipy.optimize import linprog
from collections import Counter
import itertools

In [4]:
# update front end

# TODO on algorithm
# try a heuristic split method to come up with best 
# 2 layout approach

In [ ]:
data = pd.read_excel('../decklizer/data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                           sheet_name='Schedule')

In [ ]:
df = load_schedule(data,
    customer='P & G',
    technology='SAM',
    color='WHITE',
    cycle='CYCLE 2',                 
)
# df = df.iloc[:10]

In [128]:
B = 4160
w, q, L, n = process_schedule(
    df,
    B=B,
    put_up=17000,
    doffs_in_jumbo=6,
    verbiose=True,
)

The important variables

widths: [154, 158, 170, 208, 218, 234] (mm)
neck in: [4, 4, 5, 7, 7, 7] (mm)
jumbo length (L): 102000 (m)
undeckled jumbos needed (q): [96, 32, 318, 95, 158, 461]


In [79]:
w = [205, 195, 220, 160]
n = [3, 2, 3, 1]
L = 17000
q = [776, 557, 470, 17]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [208, 158, 170, 234]
n = [2, 2, 2, 3]
L = 17000
q = [65, 120, 1001, 100]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [575, 626]
n = [4, 4]
L = 17000
q = [226, 226]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [711, 559, 889, 762, 572]
n = [16, 16, 16, 16, 16]
L = 17000
q = [20, 201, 47, 21, 10]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [208, 234, 170]
n = [2, 3, 2]
L = 17000
q = [805, 437, 226]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [218, 234, 208]
n = [2, 2, 1]
L = 17000
q = [645, 756, 141]
B = 4160
w = [i+j for i, j in zip(w,n)]

w = [930, 588, 777, 610]
lm = [8130330, 1152978, 6067685, 495868]
n = [16, 16, 16, 16]
w = [i+j for i,j in zip(w,n)]
B = 3950
L = 34000
q = [math.ceil(i/L) for i in lm]

w = [523, 629]
n = [7, 9]
L = 9900
q = [238, 449]
B = 4144

In [145]:
buckets = dict()
buckets['A'] = dict()
buckets['A']['w'] = [i+j for i, j in zip([205, 195, 220, 160],
                                         [3, 2, 3, 1])]
buckets['A']['n'] = [3, 2, 3, 1]
buckets['A']['L'] = 17000
buckets['A']['q'] = [776, 557, 470, 17]
buckets['A']['B'] = 4160

buckets['B'] = dict()
buckets['B']['w'] = [i+j for i, j in zip([575, 626, 438, 622, 749, 546],
                                         [10, 11, 9, 11, 14, 10])]
buckets['B']['n'] = [10, 11, 9, 11, 14, 10]
buckets['B']['L'] = 15000
buckets['B']['q'] = [282, 282, 5, 48, 142, 241]
buckets['B']['B'] = 3050

buckets['C'] = dict()
buckets['C']['w'] = [i+j for i, j in zip([622, 749, 800],
                                         [11, 14, 14])]
buckets['C']['n'] = [11, 14, 14]
buckets['C']['L'] = 15000
buckets['C']['q'] = [175, 495, 330]
buckets['C']['B'] = 3050

buckets['D'] = dict()
buckets['D']['w'] = [i+j for i, j in zip([616, 743],
                                         [11, 14])]
buckets['D']['n'] = [11, 14]
buckets['D']['L'] = 15000
buckets['D']['q'] = [850, 150]
buckets['D']['B'] = 3050

In [149]:
bucket = buckets['D']
max_combo=3
goal = 3
max_patterns = 3
target = 1
patterns, layout = seed_patterns(bucket['w'], 
                                 [math.ceil(i*target) 
                                  for i in bucket['q']], 
                                 bucket['B'], 
                                 bucket['n'], 
                                 max_combinations=max_combo, 
                                 goal=goal, 
                                 verbiose=verbiose)

loss, inv, summary = find_optimum(patterns, 
                                  layout, 
                                  bucket['w'], 
                                  [math.ceil(i*target) 
                                   for i in bucket['q']], 
                                  bucket['B'], 
                                  bucket['n'], 
                                  bucket['L'],
                                max_combinations=max_combo,
                                max_patterns=max_patterns,
                                prioritize='material loss')


layout pattern: {616: 3, 743: 1}
pattern loss: 13.51 %
minimum doffs to fill order: 284
inventory created: {616: 2, 743: 134}

3 possible max 3 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      13.51 % (1.76e+06 sqm)
total inventory: 11.64 % (1.51e+06 sqm)

inventory created: {616: 2.0, 743: 134.0}

layout summary:

loss: 13.51% 	 284.0 x	 {616: 3, 743: 1}

total jumbos: 284.0 (1.30e+07 sqm)


In [1]:
import multiprocessing as mp
print("Number of processors: ", mp.cpu_count())

# Parallelizing using Pool.apply()

# Step 1: Init multiprocessing.Pool()
pool = mp.Pool(mp.cpu_count())

# Step 2: `pool.apply` the `howmany_within_range()`
results = [pool.apply(howmany_within_range, args=(row, 4, 8)) for row in data]

# Step 3: Don't forget to close
pool.close()    

print(results[:10])
#> [3, 1, 4, 4, 4, 2, 1, 1, 3, 3]

Number of processors:  8


In [2]:
combination_params = [2,3]
pattern_params = [3,4]
production_targets = np.arange(0.95,1.05,.01)
edge_trim_allowance = np.arange(11)
goal = 3
verbiose = True
bucket = buckets['D']
res = []

def deckle(bucket, max_combo, max_patterns, target, edge, prioritize, goal, verbiose):
    patterns, layout = seed_patterns(bucket['w'], 
                                     [math.ceil(i*target) 
                                      for i in bucket['q']], 
                                     bucket['B']+(edge*2), 
                                     bucket['n'], 
                                     max_combinations=max_combo, 
                                     goal=goal, 
                                     verbiose=verbiose)
    loss, inv, summary = find_optimum(patterns, 
                                      layout, 
                                      bucket['w'], 
                                      [math.ceil(i*target) 
                                       for i in bucket['q']], 
                                      bucket['B']+(edge*2), 
                                      bucket['n'], 
                                      bucket['L'],
                                    max_combinations=max_combo,
                                    max_patterns=max_patterns,
                                    prioritize='time')
    return [loss, int(summary['jumbos'].sum()), 
            inv, summary, max_combo, 
            summary.shape[0], target, edge]

NameError: name 'np' is not defined

In [152]:
combination_params = [2,3]
pattern_params = [3,4]
production_targets = np.arange(0.95,1.05,.01)
edge_trim_allowance = np.arange(11)
goal = 3
verbiose = True
bucket = buckets['D']
res = []
for max_combo in combination_params:
    for max_patterns in pattern_params:
        for target in production_targets:
            for edge in edge_trim_allowance:
                patterns, layout = seed_patterns(bucket['w'], 
                                                 [math.ceil(i*target) 
                                                  for i in bucket['q']], 
                                                 bucket['B']+(edge*2), 
                                                 bucket['n'], 
                                                 max_combinations=max_combo, 
                                                 goal=goal, 
                                                 verbiose=verbiose)

                loss, inv, summary = find_optimum(patterns, 
                                                  layout, 
                                                  bucket['w'], 
                                                  [math.ceil(i*target) 
                                                   for i in bucket['q']], 
                                                  bucket['B']+(edge*2), 
                                                  bucket['n'], 
                                                  bucket['L'],
                                                max_combinations=max_combo,
                                                max_patterns=max_patterns,
                                                prioritize='time')
                res.append([loss, int(summary['jumbos'].sum()), 
                            inv, summary, max_combo, 
                            summary.shape[0], target, edge])

layout pattern: {616: 3, 743: 1}
pattern loss: 13.51 %
minimum doffs to fill order: 270
inventory created: {616: 2, 743: 127}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.21 % (1.66e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 17.77% 	 95.0 x	 {616: 4, 743: 0}
loss: 13.51% 	 143.0 x	 {616: 3, 743: 1}

total jumbos: 238.0 (1.09e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.56 %
minimum doffs to fill order: 270
inventory created: {616: 2, 743: 127}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.27 % (1.66e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 17.82% 	 95.0 x	 {616: 4, 743: 0}
loss: 13.56% 	 143.0 x	 {616: 3, 743: 1}

total jumbos: 238.0 (1.09e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.62 %
minimum doffs to 

7 unique patterns found
21 possible max 3 patterns

total loss:      15.66 % (1.73e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 18.20% 	 192.0 x	 {616: 4, 743: 0}
loss: 5.48% 	 48.0 x	 {616: 1, 743: 3}

total jumbos: 240.0 (1.10e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 14.02 %
minimum doffs to fill order: 272
inventory created: {616: 0, 743: 128}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.71 % (1.74e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 18.25% 	 192.0 x	 {616: 4, 743: 0}
loss: 5.54% 	 48.0 x	 {616: 1, 743: 3}

total jumbos: 240.0 (1.10e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 14.07 %
minimum doffs to fill order: 272
inventory created: {616: 0, 743: 128}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.77

7 unique patterns found
21 possible max 3 patterns

total loss:      15.43 % (1.73e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 17.99% 	 196.0 x	 {616: 4, 743: 0}
loss: 5.23% 	 49.0 x	 {616: 1, 743: 3}

total jumbos: 245.0 (1.12e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.79 %
minimum doffs to fill order: 278
inventory created: {616: 1, 743: 131}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.49 % (1.74e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 18.04% 	 196.0 x	 {616: 4, 743: 0}
loss: 5.29% 	 49.0 x	 {616: 1, 743: 3}

total jumbos: 245.0 (1.12e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.85 %
minimum doffs to fill order: 278
inventory created: {616: 1, 743: 131}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.55

7 unique patterns found
21 possible max 3 patterns

total loss:      15.21 % (1.74e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 17.77% 	 200.0 x	 {616: 4, 743: 0}
loss: 4.98% 	 50.0 x	 {616: 1, 743: 3}

total jumbos: 250.0 (1.14e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.56 %
minimum doffs to fill order: 284
inventory created: {616: 2, 743: 134}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.27 % (1.75e+06 sqm)
total inventory: 0.00 % (0.00e+00 sqm)

inventory created: {616: 0.0, 743: 0.0}

layout summary:

loss: 17.82% 	 200.0 x	 {616: 4, 743: 0}
loss: 5.05% 	 50.0 x	 {616: 1, 743: 3}

total jumbos: 250.0 (1.14e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.62 %
minimum doffs to fill order: 284
inventory created: {616: 2, 743: 134}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.32

7 unique patterns found
21 possible max 3 patterns

total loss:      15.60 % (1.81e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 18.15% 	 177.0 x	 {616: 4, 743: 0}
loss: 9.66% 	 76.0 x	 {616: 2, 743: 2}

total jumbos: 253.0 (1.16e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.96 %
minimum doffs to fill order: 287
inventory created: {616: 2, 743: 135}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.65 % (1.82e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 18.20% 	 177.0 x	 {616: 4, 743: 0}
loss: 9.72% 	 76.0 x	 {616: 2, 743: 2}

total jumbos: 253.0 (1.16e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 14.02 %
minimum doffs to fill order: 287
inventory created: {616: 2, 743: 135}

3 possible max 2 combinations
7 unique patterns found
21 possible max 3 patterns

total loss:      15.71

...

7 unique patterns found
total loss:      15.32 % (1.73e+06 sqm)
total inventory: 0.34 % (3.89e+04 sqm)

inventory created: {616: 3.0, 743: 1.0}

layout summary:

loss: 17.88% 	 209.0 x	 {616: 4, 743: 0}
loss: 0.85% 	 37.0 x	 {616: 0, 743: 4}

total jumbos: 246.0 (1.13e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.68 %
minimum doffs to fill order: 278
inventory created: {616: 1, 743: 131}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.37 % (1.73e+06 sqm)
total inventory: 0.34 % (3.89e+04 sqm)

inventory created: {616: 3.0, 743: 1.0}

layout summary:

loss: 17.93% 	 209.0 x	 {616: 4, 743: 0}
loss: 0.92% 	 37.0 x	 {616: 0, 743: 4}

total jumbos: 246.0 (1.13e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.73 %
minimum doffs to fill order: 278
inventory created: {616: 1, 743: 131}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.43 % (1.74e+06 sqm)
total inventory: 0.34 % (3.89e+04 sqm)

inventory created: {616: 3

7 unique patterns found
total loss:      15.19 % (1.74e+06 sqm)
total inventory: 0.36 % (4.08e+04 sqm)

inventory created: {616: 2.0, 743: 2.0}

layout summary:

loss: 17.77% 	 213.0 x	 {616: 4, 743: 0}
loss: 0.72% 	 38.0 x	 {616: 0, 743: 4}

total jumbos: 251.0 (1.15e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.56 %
minimum doffs to fill order: 284
inventory created: {616: 2, 743: 134}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.24 % (1.75e+06 sqm)
total inventory: 0.35 % (4.08e+04 sqm)

inventory created: {616: 2.0, 743: 2.0}

layout summary:

loss: 17.82% 	 213.0 x	 {616: 4, 743: 0}
loss: 0.79% 	 38.0 x	 {616: 0, 743: 4}

total jumbos: 251.0 (1.15e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.62 %
minimum doffs to fill order: 284
inventory created: {616: 2, 743: 134}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.30 % (1.76e+06 sqm)
total inventory: 0.35 % (4.08e+04 sqm)

inventory created: {616: 2

7 unique patterns found
total loss:      15.71 % (1.83e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 18.25% 	 215.0 x	 {616: 4, 743: 0}
loss: 1.30% 	 38.0 x	 {616: 0, 743: 4}

total jumbos: 253.0 (1.16e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 14.07 %
minimum doffs to fill order: 287
inventory created: {616: 2, 743: 135}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.76 % (1.84e+06 sqm)
total inventory: 0.08 % (9.24e+03 sqm)

inventory created: {616: 1.0, 743: 0.0}

layout summary:

loss: 18.31% 	 215.0 x	 {616: 4, 743: 0}
loss: 1.37% 	 38.0 x	 {616: 0, 743: 4}

total jumbos: 253.0 (1.17e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.51 %
minimum doffs to fill order: 289
inventory created: {616: 0, 743: 136}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.17 % (1.78e+06 sqm)
total inventory: 0.36 % (4.27e+04 sqm)

inventory created: {616: 1

7 unique patterns found
total loss:      15.58 % (1.85e+06 sqm)
total inventory: 0.09 % (1.11e+04 sqm)

inventory created: {616: 0.0, 743: 1.0}

layout summary:

loss: 18.15% 	 219.0 x	 {616: 4, 743: 0}
loss: 1.17% 	 39.0 x	 {616: 0, 743: 4}

total jumbos: 258.0 (1.19e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.96 %
minimum doffs to fill order: 292
inventory created: {616: 0, 743: 137}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.64 % (1.86e+06 sqm)
total inventory: 0.09 % (1.11e+04 sqm)

inventory created: {616: 0.0, 743: 1.0}

layout summary:

loss: 18.20% 	 219.0 x	 {616: 4, 743: 0}
loss: 1.24% 	 39.0 x	 {616: 0, 743: 4}

total jumbos: 258.0 (1.19e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 14.02 %
minimum doffs to fill order: 292
inventory created: {616: 0, 743: 137}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.69 % (1.86e+06 sqm)
total inventory: 0.09 % (1.11e+04 sqm)

inventory created: {616: 0

7 unique patterns found
total loss:      15.46 % (1.87e+06 sqm)
total inventory: 0.41 % (5.00e+04 sqm)

inventory created: {616: 3.0, 743: 2.0}

layout summary:

loss: 18.04% 	 224.0 x	 {616: 4, 743: 0}
loss: 1.05% 	 40.0 x	 {616: 0, 743: 4}

total jumbos: 264.0 (1.21e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.85 %
minimum doffs to fill order: 298
inventory created: {616: 1, 743: 140}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.52 % (1.88e+06 sqm)
total inventory: 0.41 % (5.00e+04 sqm)

inventory created: {616: 3.0, 743: 2.0}

layout summary:

loss: 18.09% 	 224.0 x	 {616: 4, 743: 0}
loss: 1.11% 	 40.0 x	 {616: 0, 743: 4}

total jumbos: 264.0 (1.21e+07 sqm)
layout pattern: {616: 3, 743: 1}
pattern loss: 13.90 %
minimum doffs to fill order: 298
inventory created: {616: 1, 743: 140}

3 possible max 3 combinations
7 unique patterns found
total loss:      15.57 % (1.89e+06 sqm)
total inventory: 0.41 % (5.00e+04 sqm)

inventory created: {616: 3

In [153]:
df = pd.DataFrame(res, columns=['loss', 'jumbos', 'inventory', 'summary', 
                                'combinations', 'patterns', 'target', 'edge'])
# df = df.loc[df['patterns'] < 6]
df['str summ'] = df['summary'].astype(str)
df = df.sort_values('loss').reset_index(drop=True)
df = df.drop_duplicates(['str summ'])\
    [['loss', 'jumbos', 'inventory', 'summary',
     'combinations', 'patterns', 'target', 'edge']].reset_index(drop=True)
df['loss rank'] = df['loss'].rank(pct=True)
df['jumbo rank'] = df['jumbos'].rank(pct=True)
df.shape

(242, 10)

In [131]:
gsm21bucket = df

In [161]:
df.iloc[8]['summary']

,loss,jumbos,layout
0,17.7705,95,"{616: 4, 743: 0}"
1,13.5082,143,"{616: 3, 743: 1}"


In [160]:
for i in range(10):
    print(i, df.iloc[i]['summary'])

0        loss jumbos            layout
0   17.7705    211  {616: 4, 743: 0}
1  0.721311     38  {616: 0, 743: 4}
1        loss jumbos            layout
0   17.7705    217  {616: 4, 743: 0}
1  0.721311     39  {616: 0, 743: 4}
2        loss jumbos            layout
0   17.7705    207  {616: 4, 743: 0}
1  0.721311     37  {616: 0, 743: 4}
3        loss jumbos            layout
0   17.7705    224  {616: 4, 743: 0}
1  0.721311     40  {616: 0, 743: 4}
4        loss jumbos            layout
0   17.7705    213  {616: 4, 743: 0}
1  0.721311     38  {616: 0, 743: 4}
5        loss jumbos            layout
0   17.7705    202  {616: 4, 743: 0}
1  0.721311     36  {616: 0, 743: 4}
6        loss jumbos            layout
0   17.7705    219  {616: 4, 743: 0}
1  0.721311     39  {616: 0, 743: 4}
7        loss jumbos            layout
0   17.7705    209  {616: 4, 743: 0}
1  0.721311     37  {616: 0, 743: 4}
8       loss jumbos            layout
0  17.7705     95  {616: 4, 743: 0}
1  13.5082    143  {61

In [155]:
df

,loss,jumbos,inventory,summary,combinations,patterns,target,edge,loss rank,jumbo rank
0,15.168609,249,"{616: 2.0, 743: 3.0}",loss jumbos layout 0 17.77...,2,2,0.99,0,0.004132,0.433884
1,15.173156,256,"{616: 1.0, 743: 3.0}",loss jumbos layout 0 17.77...,3,2,1.02,0,0.008264,0.706612
2,15.185165,244,"{616: 3.0, 743: 2.0}",loss jumbos layout 0 17.77...,2,2,0.97,0,0.012397,0.252066
3,15.187283,264,"{616: 3.0, 743: 2.0}",loss jumbos layout 0 17.77...,2,2,1.05,0,0.016529,0.979339
4,15.189341,251,"{616: 2.0, 743: 2.0}",loss jumbos layout 0 17.77...,2,2,1.00,0,0.020661,0.524793
...,...,...,...,...,...,...,...,...,...,...
237,15.765472,240,"{616: 0.0, 743: 0.0}",loss jumbos layout 0 18.3062...,3,2,0.96,10,0.987603,0.138430
238,15.765472,250,"{616: 0.0, 743: 0.0}",loss jumbos layout 0 18.3062...,2,2,1.00,10,0.987603,0.479339
239,15.765472,245,"{616: 0.0, 743: 0.0}",loss jumbos layout 0 18.3062...,2,2,0.98,10,0.987603,0.297521
240,15.765472,260,"{616: 0.0, 743: 0.0}",loss jumbos layout 0 18.3062...,3,2,1.04,10,0.997934,0.865702
